In [ ]:
import json
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
def train_and_save_models(country: str):
    # Load events data from Wyscout
    with open(f'wyscout_data/raw/events_{country}.json', 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    df = df[df['positions'].apply(lambda x: len(x) == 2)]

    # Normalize position data
    df['x0'] = df['positions'].apply(lambda x: x[0]['x'])
    df['y0'] = df['positions'].apply(lambda x: x[0]['y'])
    df['x1'] = df['positions'].apply(lambda x: x[1]['x'])
    df['y1'] = df['positions'].apply(lambda x: x[1]['y'])

    # Add goal and shot columns
    df['goal'] = df['tags'].apply(lambda tags: 1 if any(tag['id'] == 101 for tag in tags) else 0)
    df['is_shot'] = df['eventName'].apply(lambda event: 1 if event == 'Shot' else 0)

    # Identify possession chains
    df['team'] = df['teamId']
    df['player'] = df['playerId']

    # Create a possession ID to group events by possession
    df['possession_id'] = (df['team'] != df['team'].shift()).cumsum()

    # Create a new column indicating whether the possession ended in a shot
    df['end_with_shot'] = df.groupby('possession_id')['is_shot'].transform('max')

    # Prepare features and target variables for shot probability (XGBoost Classifier)
    feature_columns = ['x0', 'y0', 'x1', 'y1']
    X_possession = df[feature_columns]
    y_possession = df['end_with_shot']

    # Split the data into training and testing sets
    X_train_possession, X_test_possession, y_train_possession, y_test_possession = train_test_split(
        X_possession, y_possession, test_size=0.2, random_state=42
    )

    # XGBoost classifier for predicting shot probability
    possession_model = xgb.XGBClassifier()
    possession_model.fit(X_train_possession, y_train_possession)

    # Filter for shots to train the xG model
    shots_df = df[df['is_shot'] == 1]
    X_xG = shots_df[['x0', 'y0']]  # Only take x0 and y0 as features for xG
    y_xG = shots_df['goal']

    # Split xG data into training and testing sets
    X_train_xG, X_test_xG, y_train_xG, y_test_xG = train_test_split(X_xG, y_xG, test_size=0.2, random_state=42)

    # XGBoost regressor for predicting xG
    xG_model = xgb.XGBRegressor()
    xG_model.fit(X_train_xG, y_train_xG)

    # Save the trained models
    joblib.dump(possession_model, f'models/{country}_possession_model.pkl')
    joblib.dump(xG_model, f'models/{country}_xG_model.pkl')

    print(f"Models saved for {country}: possession_model.pkl and xG_model.pkl")


In [ ]:
def calculate_defensive_contribution(country: str):
    # Load players JSON file
    with open('wyscout_data/raw/players.json', 'r') as file:
        players_data = json.load(file)

    # Load matches data
    with open(f'wyscout_data/raw/matches_{country}.json', 'r') as file:
        data = json.load(file)

    # Initialize a dictionary for team mapping
    team_mapping = {}

    # Extract team data from the JSON structure
    for match in data:
        teams = match.get('teamsData', {})
        for team_id, team_info in teams.items():
            players = team_info.get('formation', {}).get('lineup', []) + team_info.get('formation', {}).get('bench', [])
            for player in players:
                player_id = player['playerId']
                team_mapping[player_id] = int(team_id)

    # Load events data from Wyscout
    with open(f'wyscout_data/raw/events_{country}.json', 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    df = df[df['positions'].apply(lambda x: len(x) == 2)]

    # Normalize position data
    df['x0'] = df['positions'].apply(lambda x: x[0]['x'])
    df['y0'] = df['positions'].apply(lambda x: x[0]['y'])
    df['x1'] = df['positions'].apply(lambda x: x[1]['x'])
    df['y1'] = df['positions'].apply(lambda x: x[1]['y'])

    # Add goal and shot columns
    df['goal'] = df['tags'].apply(lambda tags: 1 if any(tag['id'] == 101 for tag in tags) else 0)
    df['is_shot'] = df['eventName'].apply(lambda event: 1 if event == 'Shot' else 0)

    # Load the pre-trained models
    possession_model = joblib.load(f'models/{country}_possession_model.pkl')
    xG_model = joblib.load(f'models/{country}_xG_model.pkl')

    # Predict shot probability for the entire dataset
    feature_columns = ['x0', 'y0', 'x1', 'y1']
    X_possession = df[feature_columns]
    df['shot_prob'] = possession_model.predict_proba(X_possession)[:, 1]

    # Filter for shots to predict xG
    df['xG'] = 0
    feature_columns_Xg = ['x0', 'y0']
    df.loc[df['is_shot'] == 1, 'xG'] = xG_model.predict(df.loc[df['is_shot'] == 1, feature_columns_Xg])

    # Calculate expected threat (xT) as the product of shot probability and xG
    df['xT'] = df['shot_prob'] * df['xG']

    # --------------------- Defensive Contribution Calculation ---------------------
       # Define defensive action types
    defensive_action_types = {
        'Tackle/Duel': {
            'eventName': 'Duel',
            'subEventNames': ['Air duel', 'Ground loose ball duel', 'Ground defending duel']
        },
        'Interception': {
            'eventName': 'Duel',
            'subEventNames': ['Ground defending duel']
        },
        'Clearance': {
            'eventName': 'Others on the ball',
            'subEventNames': ['Clearance']
        },
        'Block/Save': {
            'eventName': 'Save attempt',
            'subEventNames': ['Save attempt', 'Shot']
        }
    }

    # Extract defensive actions in the defensive third
    defensive_data = []
    for index, row in df.iterrows():
        player_id = row['playerId']
        event_name = row['eventName']
        sub_event_name = row['subEventName']
        team_id = row['teamId']  # Team performing the defensive action
        goal_scored = row['goal']

        # Check for defensive actions based on defined types
        for action_type, conditions in defensive_action_types.items():
            if event_name == conditions['eventName'] and sub_event_name in conditions['subEventNames']:
                defensive_data.append({
                    'playerId': player_id,
                    'teamId': team_id,
                    'eventName': event_name,
                    'subEventName': sub_event_name,
                    'matchId': row['matchId'],
                    'eventSec': row['eventSec'],
                    'matchPeriod': row['matchPeriod'],
                    'x0': row['x0'],
                    'y0': row['y0'],
                    'x1': row['x1'],
                    'y1': row['y1'],
                    'actionType': action_type,
                    'is_goal': goal_scored  # Flag for whether the action resulted in a goal
                })

    

    defensive_df = pd.DataFrame(defensive_data)

    # Debug: print the defensive DataFrame
    print(defensive_df)

    defensive_df = defensive_df[defensive_df['x0'] <= 33.33]  # Filter for actions in the defensive third

    # Debug: print the defensive DataFrame
    print(defensive_df)

    defensive_df = defensive_df.reset_index(drop=True)

    # Debug: print the defensive DataFrame
    print(defensive_df)

    # Calculate xT before the defensive action using transformed coordinates
    defensive_df['x0_transformed'] = 100 - defensive_df['x0']
    defensive_df['y0_transformed'] = 100 - defensive_df['y0']
    defensive_df['x1_transformed'] = 100 - defensive_df['x1']
    defensive_df['y1_transformed'] = 100 - defensive_df['y1']

    # Calculate xT before the defensive action
    defensive_df['xT_before'] = possession_model.predict_proba(defensive_df[['x0', 'y0', 'x1', 'y1']].apply(lambda x: 100 - x))[:, 1] * \
                                xG_model.predict(defensive_df[['x0', 'y0']].apply(lambda x: 100 - x))

    # Calculate xT_after
    xT_after_values = []

    for index, row in defensive_df.iterrows():
        match_id = row['matchId']
        player_id = row['playerId']
        defensive_event_time = row['eventSec']
        match_period = row['matchPeriod']

        # Skip if player_id is 0 (invalid player data)
        if player_id == 0:
            xT_after_values.append(0)
            continue

        # Get the next two touches after the defensive event
        next_two_touches = df[(df['matchId'] == match_id) & (df['eventSec'] > defensive_event_time)].head(2)
        defending_team_id = team_mapping[player_id]  # Defensive team ID

        # Find the offensive team touches (team that is NOT defending)
        offensive_team_touches = next_two_touches[next_two_touches['teamId'] != defending_team_id]

        # Check if the ball is out of play as a result of the defensive action
        if (row['x1'] == 0 and row['y1'] == 0) or (row['x1'] == 100 and row['y1'] == 100):
            xT_after_values.append(0)
            continue

        if not offensive_team_touches.empty:
            # Check if this defensive action resulted in a goal
            if row['is_goal'] == 1:
                # If the defensive action led to a goal, xT_after should be 0 (since it's a negative result)
                xT_after_values.append(defensive_df['xT_before'].iloc[index])
            else:
                # Calculate xT_after for offensive team touches
                after_action_xT = possession_model.predict_proba(offensive_team_touches[['x0', 'y0', 'x1', 'y1']].apply(lambda x: 100 - x))[:, 1] * \
                                xG_model.predict(offensive_team_touches[['x0', 'y0']].apply(lambda x: 100 - x))

                # Append the mean of xT after offensive touches
                xT_after_values.append(after_action_xT.mean())
        else:
            # If no offensive team touches are found, append 0 for xT_after
            xT_after_values.append(0)


    defensive_df['xT_after'] = xT_after_values
    defensive_df['xT_difference'] = defensive_df['xT_before'] - defensive_df['xT_after']

    return defensive_df



In [ ]:
def calculate_defensive_summary(country: str, dci):
    # Load players JSON file
    with open('wyscout_data/raw/players.json', 'r') as file:
        players = json.load(file)
    players_df = pd.json_normalize(players)

    # Load matches data
    with open(f'wyscout_data/raw/matches_{country}.json', 'r') as file:
        data = json.load(file)

    # Initialize a dictionary for team mapping
    team_mapping = {}

    # Extract team data from the JSON structure
    for match in data:
        teams = match.get('teamsData', {})
        for team_id, team_info in teams.items():
            players = team_info.get('formation', {}).get('lineup', []) + team_info.get('formation', {}).get('bench', [])
            for player in players:
                player_id = player['playerId']
                team_mapping[player_id] = int(team_id)

    # Load events data from Wyscout
    with open(f'wyscout_data/raw/events_{country}.json', 'r') as file:
        data = json.load(file)

    df = pd.json_normalize(data)
    df = df[df['positions'].apply(lambda x: len(x) == 2)]

    # Normalize position data
    df['x0'] = df['positions'].apply(lambda x: x[0]['x'])
    df['y0'] = df['positions'].apply(lambda x: x[0]['y'])
    df['x1'] = df['positions'].apply(lambda x: x[1]['x'])
    df['y1'] = df['positions'].apply(lambda x: x[1]['y'])

    # Add goal and shot columns
    df['goal'] = df['tags'].apply(lambda tags: 1 if any(tag['id'] == 101 for tag in tags) else 0)
    df['is_shot'] = df['eventName'].apply(lambda event: 1 if event == 'Shot' else 0)

    # Load the pre-trained models
    possession_model = joblib.load(f'models/{country}_possession_model.pkl')
    xG_model = joblib.load(f'models/{country}_xG_model.pkl')

    # Predict shot probability for the entire dataset
    feature_columns = ['x0', 'y0', 'x1', 'y1']
    X_possession = df[feature_columns]
    df['shot_prob'] = possession_model.predict_proba(X_possession)[:, 1]

    # Filter for shots to predict xG
    df['xG'] = 0
    df.loc[df['is_shot'] == 1, 'xG'] = xG_model.predict(df.loc[df['is_shot'] == 1, ['x0', 'y0']])  # Use only x0 and y0 for xG

    # Calculate expected threat (xT) as the product of shot probability and xG
    df['xT'] = df['shot_prob'] * df['xG']

    # Load players JSON file
    with open('wyscout_data/raw/players.json', 'r') as file:
        players = json.load(file)
    players_df = pd.json_normalize(players)

    # Load teams JSON file
    with open('wyscout_data/raw/teams.json', 'r') as file:
        teams = json.load(file)
    teams_df = pd.json_normalize(teams)

    # Calculate defensive summary
    defensive_summary = (
        dci[['playerId', 'teamId', 'xT_difference']]
        .groupby(['playerId', 'teamId'])
        .sum()
        .reset_index()
    )

    # Calculate total defensive actions
    defensive_actions_count = dci.groupby('playerId').size().reset_index(name='defensive_actions')
    defensive_summary = defensive_summary.merge(defensive_actions_count, on='playerId', how='left')

    # Check for NaN values in defensive_actions
    print("Defensive actions count:\n", defensive_actions_count)
    print("Defensive summary before DCI calculation:\n", defensive_summary)

    # Calculate total Defensive Contribution Index (DCI)
    alpha = 1.0
    beta = 1.0
    defensive_summary['DCI'] = alpha * defensive_summary['defensive_actions'] + beta * defensive_summary['xT_difference']

    # Merge with playing time data and calculate DCI per 90 minutes
    player_total_time = df.groupby(['playerId', 'matchId'])['eventSec'].max().groupby('playerId').sum().reset_index(name='total_time')
    player_total_time['total_time'] = player_total_time['total_time'] / 60  # Convert to minutes

    # Merge defensive summary with total time
    defensive_summary = defensive_summary.merge(player_total_time, on='playerId', how='left')

    # Calculate DCI per 90 minutes
    defensive_summary['DCI_per_90'] = defensive_summary['DCI'] / (defensive_summary['total_time'] / 90)

    # Handle NaN values in DCI
    defensive_summary['DCI'].fillna(0, inplace=True)

    # Check for NaN values in the final summary
    print("Final Defensive Summary:\n", defensive_summary)
    print("NaN values count in DCI:", defensive_summary['DCI'].isna().sum())

    defensive_summary['name'] = defensive_summary['playerId'].map(players_df.set_index('wyId')['shortName'])
    defensive_summary['team'] = defensive_summary['teamId'].map(teams_df.set_index('wyId')['name'])

    defensive_summary = defensive_summary[defensive_summary['playerId'] != 0].sort_values(by='DCI_per_90', ascending=False)

    return defensive_summary


In [ ]:
countries = [ 'Spain', 'Germany', 'Italy', 'France', 'England',]
contributions1 = []

for country in countries:
    train_and_save_models(country)
    dci = calculate_defensive_contribution(country)
    contributions1.append(calculate_defensive_summary(country, dci))

    for i in range(len(contributions1)):
    
        contributions1[i]['defensive_actions_divided'] = contributions1[i]['defensive_actions']  * 0.01
        contributions1[i]['new_DCI'] = contributions1[i]['xT_difference'] + contributions1[i]['defensive_actions_divided'] 
        contributions1[i]['new_DCI_per_90'] = contributions1[i]['new_DCI'] / (contributions1[i]['total_time'] / 90)


        contributions1[i].to_csv(f'{countries[i]}_defensive_summary.csv', index=False)

Models saved for Spain: possession_model.pkl and xG_model.pkl


C:\Users\ondre\AppData\Local\Temp\ipykernel_3360\905492943.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.01125083 0.19740705 0.07353181 ... 0.4494221  0.00278937 0.11430936]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['is_shot'] == 1, 'xG'] = xG_model.predict(df.loc[df['is_shot'] == 1, feature_columns_Xg])


        playerId  teamId           eventName            subEventName  matchId  \
0           3542     682                Duel   Ground defending duel  2565548   
1           3542     682                Duel   Ground defending duel  2565548   
2           3695     682                Duel   Ground defending duel  2565548   
3           3695     682                Duel   Ground defending duel  2565548   
4           6914     695                Duel  Ground loose ball duel  2565548   
...          ...     ...                 ...                     ...      ...   
183739     69404     675                Duel   Ground defending duel  2565927   
183740     69404     675                Duel   Ground defending duel  2565927   
183741    122832     682                Duel   Ground defending duel  2565927   
183742    122832     682                Duel   Ground defending duel  2565927   
183743      3306     675  Others on the ball               Clearance  2565927   

           eventSec matchPe

C:\Users\ondre\AppData\Local\Temp\ipykernel_3360\3223099082.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.01125083 0.19740705 0.07353181 ... 0.4494221  0.00278937 0.11430936]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['is_shot'] == 1, 'xG'] = xG_model.predict(df.loc[df['is_shot'] == 1, ['x0', 'y0']])  # Use only x0 and y0 for xG
C:\Users\ondre\AppData\Local\Temp\ipykernel_3360\3223099082.py:102: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead

Defensive actions count:
      playerId  defensive_actions
0           0               7665
1          33                  3
2          99                  1
3         151                 38
4         254                494
..        ...                ...
534    500200                  7
535    516397                 16
536    519496                 14
537    520163                 13
538    551398                 20

[539 rows x 2 columns]
Defensive summary before DCI calculation:
      playerId  teamId  xT_difference  defensive_actions
0           0     674       2.581464               7665
1           0     675       2.163161               7665
2           0     676       1.398579               7665
3           0     677       2.779390               7665
4           0     678       3.437411               7665
..        ...     ...            ...                ...
569    500200     698       0.035065                  7
570    516397     677       0.298948                 16
571    

...

C:\Users\ondre\AppData\Local\Temp\ipykernel_3360\905492943.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.3923017  0.32365683 0.47135395 ... 0.05415696 0.25297666 0.119463  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['is_shot'] == 1, 'xG'] = xG_model.predict(df.loc[df['is_shot'] == 1, feature_columns_Xg])


        playerId  teamId     eventName            subEventName  matchId  \
0         253784    3799          Duel                Air duel  2500686   
1          56441    3772          Duel                Air duel  2500686   
2          28529    3772          Duel   Ground defending duel  2500686   
3          28529    3772          Duel   Ground defending duel  2500686   
4         366760    3799          Duel  Ground loose ball duel  2500686   
...          ...     ...           ...                     ...      ...   
183276     28922   19830          Duel   Ground defending duel  2501065   
183277     28922   19830          Duel   Ground defending duel  2501065   
183278    393284    3795          Duel   Ground defending duel  2501065   
183279    393284    3795          Duel   Ground defending duel  2501065   
183280     25521    3795  Save attempt            Save attempt  2501065   

           eventSec matchPeriod  x0  y0  x1  y1    actionType  is_goal  
0          6.827043       

C:\Users\ondre\AppData\Local\Temp\ipykernel_3360\3223099082.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.3923017  0.32365683 0.47135395 ... 0.05415696 0.25297666 0.119463  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['is_shot'] == 1, 'xG'] = xG_model.predict(df.loc[df['is_shot'] == 1, ['x0', 'y0']])  # Use only x0 and y0 for xG


Defensive actions count:
      playerId  defensive_actions
0           0               7867
1         102                496
2         118                 51
3         123                 23
4         132                 42
..        ...                ...
515    497355                  2
516    498357                 38
517    499295                 11
518    521939                  2
519    566942                 18

[520 rows x 2 columns]
Defensive summary before DCI calculation:
      playerId  teamId  xT_difference  defensive_actions
0           0    3766       2.982459               7867
1           0    3767       2.233254               7867
2           0    3770       2.182151               7867
3           0    3771       1.856868               7867
4           0    3772       1.435525               7867
..        ...     ...            ...                ...
543    497355    3766       0.053140                  2
544    498357    3777       0.192136                 38
545    

C:\Users\ondre\AppData\Local\Temp\ipykernel_3360\3223099082.py:102: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  defensive_summary['DCI'].fillna(0, inplace=True)


In [28]:
country = 'Spain'
country_df = pd.read_csv(f'defensive_summaries/{country}_defensive_summary.csv')
# Function to decode JSON Unicode escape sequences like \u00e9 to é
def fix_unicode_escape(s):
    try:
        # Decode Unicode escape sequences (like \u00e9 -> é)
        return s.encode('utf-8').decode('unicode_escape')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # If something goes wrong, return the string as is
        return s

# Apply the fix to the 'name' list
country_df['name'] = [
    fix_unicode_escape(x) if isinstance(x, str) else x for x in country_df['name']
]

country_df['team'] = [
    fix_unicode_escape(x) if isinstance(x, str) else x for x in country_df['team']
]
display(country_df[country_df['total_time'] > 200].sort_values(by='new_DCI_per_90', ascending=False).head(20).reset_index(drop=True))

,playerId,teamId,xT_difference,defensive_actions,DCI,total_time,DCI_per_90,name,team,defensive_actions_divided,new_DCI,new_DCI_per_90
0,101649,680,0.985466,91,91.985466,229.977611,35.997817,N. Pareja,Sevilla,0.91,1.895466,0.741776
1,3885,698,1.577854,133,134.577854,371.771209,32.579196,L. Cabrera,Getafe,1.33,2.907854,0.703946
2,3335,684,2.727074,290,292.727074,756.534241,34.823852,Bartra,Real Betis,2.90,5.627074,0.669417
3,224080,696,2.704526,366,368.704526,889.327338,37.312928,G. Maripán,Deportivo Alavés,3.66,6.364526,0.644091
4,50102,677,3.886840,440,443.886840,1162.214763,34.373867,F. Schär,Deportivo La Coruña,4.40,8.286840,0.641719
5,282441,675,1.284567,89,90.284567,320.075741,25.386526,Jesús Vallejo,Real Madrid,0.89,2.174567,0.611452
6,250805,698,4.571413,659,663.571413,1712.620450,34.871373,Djené,Getafe,6.59,11.161413,0.586544
7,3468,698,2.655773,342,344.655773,949.291164,32.675981,Juan Cala,Getafe,3.42,6.075773,0.576029
8,211853,674,2.281408,334,336.281408,886.673185,34.133576,Rúben Vezo,Valencia,3.34,5.621408,0.570590
9,20808,712,2.076932,326,328.076932,844.443076,34.966151,E. Muñoz,Leganés,3.26,5.336932,0.568806
